In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np, os
from scripts.core import *
from fastai.learner import Learner
import fastai
from torch.utils.data import Dataset
from scripts.models import *
from scripts.learners import modelLearner, ParallelLearner

In [3]:
class Vid100Datasat(Dataset):
    def __init__(self, train=True, trim_start=50, trim_end=50):
        super().__init__()
        import random
        self.train=train
        self.trim_start,self.trim_end = trim_start,trim_end
        PTH = '/home/sufiyan/Common_data/mtp2/makeDataset/Sub Video/data/Mercedes_224_224_Manual_annonate_8sec_LSTM_short_005_'
        PTH_L=os.path.join(PTH,"LEFT")
        PTH_R=os.path.join(PTH,"RIGHT")
        PTH_C=os.path.join(PTH,"CLEAR")
        left_vids =[os.path.join(d,f) for d,_,fs in os.walk(PTH_L) for f in fs if f.split(".")[-1]=='npy']
        right_vids=[os.path.join(d,f) for d,_,fs in os.walk(PTH_R) for f in fs if f.split(".")[-1]=='npy']
        clear_vids=[os.path.join(d,f) for d,_,fs in os.walk(PTH_C) for f in fs if f.split(".")[-1]=='npy']
        self.fileNames=left_vids+right_vids
        self.fileNames=[filename for filename in self.fileNames if self.get_label(filename)[100]!="CLEAR"]
        np.random.shuffle(self.fileNames)
     
    @staticmethod
    def stoi(label):
        stoi={"CLEAR":0, "LEFT":1, "RIGHT":2}
        return stoi[label]
    
    def __getitem__(self,i):
        X=np.load(self.fileNames[i])[self.trim_start:-self.trim_end]
        y=self.get_label(self.fileNames[i])[self.trim_start:-self.trim_end]
        y=np.asarray([self.stoi(label) for label in y])
        X=np.expand_dims(X,1)
        X=np.concatenate((X,X,X), axis=1)
        return (torch.from_numpy(X), y)
    def __len__(self): return len(self.fileNames)
    
    def get_label(self, filename):
#         filename=self.fileNames[i]
        if self.train:
              return np.load(filename.replace('/data/','/label/'))
        else: return np.load(filename.replace('/data/','/val_label/'))

In [4]:
trainLoader=torch.utils.data.DataLoader(Vid100Datasat(train=True) ,batch_size=1, shuffle=True)
validLoader=torch.utils.data.DataLoader(Vid100Datasat(train=False),batch_size=1, shuffle=True)

In [5]:
learner=ParallelLearner([modelLearner(C3D_resnet_ConvLSTM2D(3, 3), loss_fn=nn.MSELoss, optim=torch.optim.SGD, modelName="MSE_22Oct", lr=0.01, is_depth=True)], trainLoader=trainLoader, validLoader=validLoader, printEvery=1)

In [6]:
learner.train(epochs=3)

Avg Loss: 0.248651596294208: 100%|██████████| 132/132 [01:34<00:00,  1.40it/s]  



**************************************************
Epoch: 0   Time Elapsed: 94.49109148979187
lr: 0.01      trainLoss: 0.248651596294208


  0%|          | 0/132 [00:00<?, ?it/s]

lr: 0.01      MSE validationLoss: 0.19941101629625668


Avg Loss: 0.1978953516844547: 100%|██████████| 132/132 [01:34<00:00,  1.40it/s] 



**************************************************
Epoch: 1   Time Elapsed: 241.57834768295288
lr: 0.01      trainLoss: 0.1978953516844547


  0%|          | 0/132 [00:00<?, ?it/s]

lr: 0.01      MSE validationLoss: 0.18487916661031317


Avg Loss: 0.1888703667191845: 100%|██████████| 132/132 [01:39<00:00,  1.33it/s] 



**************************************************
Epoch: 2   Time Elapsed: 396.8367838859558
lr: 0.01      trainLoss: 0.1888703667191845
lr: 0.01      MSE validationLoss: 0.18184399418532848


In [22]:
learner.train(1)

Avg Loss: 0.1852452741992293: 100%|██████████| 132/132 [01:34<00:00,  1.40it/s] 



**************************************************
Epoch: 0   Time Elapsed: 94.41840195655823
lr: 0.01      trainLoss: 0.1852452741992293
lr: 0.01      MSE validationLoss: 0.1803826458300605


In [28]:
c=learner.learners[0].valid_confusion_matrix_list

#### The Model has overfitted to Clear Label

In [29]:
c[-1]

array([[8132,    0,    0],
       [2534,    0,    0],
       [2534,    0,    0]], dtype=int32)